In [4]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1500)
pd.options.display.max_colwidth = 100000

#### Para testear las funciones, levanto los dataframes originales

In [6]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
df_adoption = pd.read_csv("./data/conversiones.csv")
year = 2018
mes_snapshot = 9

def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

universo = universo_train(df_pageviews)

In [7]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    temp = _df_refactor(temp)
    return temp

In [8]:
def _df_refactor(df_temp):
    df_union = pd.merge(universo, df_temp, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = df_union.drop(['snapshot_mes'], axis=1)
    df_union = df_union.sort_values('user_id')
    return df_union

In [9]:
def _sum_total_user(df_pageviews, campo):
    temp = df_pageviews.groupby("USER_ID")[campo].sum().to_frame().reset_index()
    temp = _df_refactor(temp)
    temp = temp.drop(['USER_ID'], axis=1)
    return temp

In [10]:
def _df_complete_columns(df_complete_original, df_incomplete):
    df_complete = df_complete_original.copy()
    df_complete[:] = 0
    df = pd.DataFrame()
    for a_column in df_complete:
        if a_column in df_incomplete.columns:
            df[a_column] = df_incomplete[a_column]
        else:
            df[a_column] = df_complete[a_column]
    return df

In [11]:
def _df_repeat_columns(df_complete_original, df_incomplete):
    users = df_complete_original['user_id']
    df_complete = df_complete_original
    df = df_incomplete[[df_incomplete.columns[1] for i in range(df_complete.shape[1]-1)]]
    df['user_id'] = users
    return df

In [12]:
def _universo_last_adoption_month(universo, df_adoption):
    df_last_adoption = df_adoption.groupby('USER_ID')[('mes','anio')].max().reset_index()
    return df_last_adoption

In [13]:
def _get_valid_pageviews_last_adoption(universo, df_pageviews, df_adoption):
    df_last_adoption = _universo_last_adoption_month(universo, df_adoption)
    df_last_adoption_page = pd.merge(df_last_adoption, df_pageviews, how='left', left_on=['USER_ID'], right_on=['USER_ID'])

    df_last_adoption_page['last_adoption_month'] = pd.to_datetime(dict(year=df_last_adoption_page.anio, month=df_last_adoption_page.mes, day=1))
    df_last_adoption_page['valid_data_from'] = df_last_adoption_page.last_adoption_month + pd.DateOffset(months=-1)
    df_last_adoption_page['valid_data_to'] = df_last_adoption_page.last_adoption_month
    
    df_last_adoption_page = df_last_adoption_page[df_last_adoption_page['FEC_EVENT'] >= df_last_adoption_page['valid_data_from']]
    df_last_adoption_page = df_last_adoption_page[df_last_adoption_page['FEC_EVENT'] < df_last_adoption_page['valid_data_to']]
    df_last_adoption_page = pd.merge(universo, df_last_adoption_page, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_last_adoption_page.loc[:, df_last_adoption_page.columns != 'USER_ID']
    return df_last_adoption_page

In [14]:
def _get_valid_pageviews_last_month(universo, df_pageviews, mes_snapshot):
    df_last_month = pd.merge(universo, df_pageviews, how='left', left_on=['user_id'], right_on=['USER_ID'])
    
    df_last_month['valid_data_to'] = pd.to_datetime(dict(year=year, month=df_last_month.snapshot_mes, day=1))
    df_last_month['valid_data_from'] = df_last_month.valid_data_to + pd.DateOffset(months=-1)
    
    df_last_month = df_last_month[df_last_month['FEC_EVENT'] >= df_last_month['valid_data_from']]
    df_last_month = df_last_month[df_last_month['FEC_EVENT'] < df_last_month['valid_data_to']]
    df_last_month = pd.merge(universo, df_last_month, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_last_month.loc[:, df_last_month.columns != 'USER_ID']
    return df_last_month

In [15]:
def _all_pageview_last_adoption_and_last_month(universo, df_pageviews, df_adoption, mes_snapshot, campo):
    pageviews_last_adoption = _get_valid_pageviews_last_adoption(universo, df_pageviews, df_adoption)
    pageviews_last_adoption_sum = _sum_campo_user(pageviews_last_adoption, campo)
    pageviews_last_adoption_sum_tot = _sum_total_user(pageviews_last_adoption, campo)
    
    pageviews_last_month = _get_valid_pageviews_last_month(universo, df_pageviews, mes_snapshot)
    pageviews_last_month_sum = _sum_campo_user(pageviews_last_month, campo)
    pageviews_last_month_sum_tot = _sum_total_user(pageviews_last_month, campo)
    
    return pageviews_last_adoption, pageviews_last_month, pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot

In [16]:
campo = 'PAGE'
pageviews_last_adoption, pageviews_last_month, pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot= _all_pageview_last_adoption_and_last_month(universo, df_pageviews, df_adoption, mes_snapshot, campo)


In [19]:
pageviews_last_adoption_sum.head(10)

,user_id,PAGE_1.0,PAGE_2.0,PAGE_3.0,PAGE_4.0,PAGE_5.0,PAGE_6.0,PAGE_7.0,PAGE_8.0,PAGE_9.0,PAGE_10.0,PAGE_11.0,PAGE_12.0,PAGE_13.0,PAGE_14.0,PAGE_15.0,PAGE_16.0,PAGE_17.0,PAGE_18.0,PAGE_19.0,PAGE_20.0,PAGE_21.0,PAGE_22.0,PAGE_23.0,PAGE_24.0,PAGE_25.0,PAGE_26.0,PAGE_27.0,PAGE_28.0,PAGE_29.0,PAGE_30.0,PAGE_31.0,PAGE_32.0,PAGE_33.0,PAGE_34.0,PAGE_36.0,PAGE_39.0,PAGE_40.0,PAGE_41.0,PAGE_42.0,PAGE_43.0,PAGE_44.0,PAGE_45.0,PAGE_46.0,PAGE_47.0,PAGE_48.0,PAGE_49.0,PAGE_50.0,PAGE_51.0,PAGE_52.0,PAGE_53.0,PAGE_54.0,PAGE_55.0,PAGE_56.0,PAGE_57.0,PAGE_58.0,PAGE_59.0,PAGE_60.0,PAGE_61.0,PAGE_62.0,PAGE_63.0,PAGE_64.0,PAGE_65.0,PAGE_66.0,PAGE_67.0,PAGE_68.0,PAGE_69.0,PAGE_71.0,PAGE_72.0,PAGE_73.0,PAGE_74.0,PAGE_75.0,PAGE_76.0,PAGE_77.0,PAGE_78.0,PAGE_79.0,PAGE_80.0,PAGE_81.0,PAGE_82.0,PAGE_83.0,PAGE_84.0,PAGE_85.0,PAGE_86.0,PAGE_87.0,PAGE_88.0,PAGE_89.0,PAGE_90.0,PAGE_91.0,PAGE_92.0,PAGE_93.0,PAGE_94.0,PAGE_95.0,PAGE_96.0,PAGE_97.0,PAGE_98.0,PAGE_99.0,PAGE_100.0,PAGE_102.0,PAGE_103.0,PAGE_104.0,PAGE_105.0,PAGE_106.0,PAGE_107.0,PAGE_108.0,PAGE_109.0,PAGE_110.0,PAGE_111.0,PAGE_113.0,PAGE_114.0,PAGE_115.0,PAGE_116.0,PAGE_118.0,PAGE_119.0,PAGE_120.0,PAGE_121.0,PAGE_122.0,PAGE_123.0,PAGE_126.0,PAGE_127.0,PAGE_128.0,PAGE_129.0,PAGE_130.0,PAGE_131.0,PAGE_132.0,PAGE_133.0,PAGE_135.0,PAGE_136.0,PAGE_137.0,PAGE_138.0,PAGE_139.0,PAGE_140.0,PAGE_141.0,PAGE_142.0,PAGE_143.0,PAGE_144.0,PAGE_145.0,PAGE_146.0,PAGE_150.0,PAGE_151.0,PAGE_152.0,PAGE_153.0,PAGE_154.0,PAGE_155.0,PAGE_156.0,PAGE_157.0,PAGE_158.0,PAGE_159.0,PAGE_160.0,PAGE_161.0,PAGE_162.0,PAGE_163.0,PAGE_164.0,PAGE_165.0,PAGE_166.0,PAGE_167.0,PAGE_168.0,PAGE_169.0,PAGE_170.0,PAGE_171.0,PAGE_172.0,PAGE_173.0,PAGE_174.0,PAGE_175.0,PAGE_176.0,PAGE_177.0,PAGE_178.0,PAGE_179.0,PAGE_180.0,PAGE_181.0,PAGE_182.0,PAGE_183.0,PAGE_184.0,PAGE_185.0,PAGE_186.0,PAGE_187.0,PAGE_188.0,PAGE_189.0,PAGE_190.0,PAGE_191.0,PAGE_192.0,PAGE_193.0,PAGE_195.0,PAGE_196.0,PAGE_197.0,PAGE_198.0,PAGE_199.0,PAGE_200.0,PAGE_202.0,PAGE_203.0,PAGE_204.0,PAGE_205.0,PAGE_206.0,PAGE_207.0,PAGE_209.0,PAGE_210.0,PAGE_211.0,PAGE_212.0,PAGE_213.0,PAGE_214.0,PAGE_215.0,PAGE_217.0,PAGE_218.0,PAGE_219.0,PAGE_220.0,PAGE_221.0,PAGE_222.0,PAGE_223.0,PAGE_224.0,PAGE_225.0,PAGE_226.0,PAGE_227.0,PAGE_228.0,PAGE_229.0,PAGE_230.0,PAGE_231.0,PAGE_232.0,PAGE_233.0,PAGE_234.0,PAGE_235.0,PAGE_236.0,PAGE_237.0,PAGE_238.0,PAGE_239.0,PAGE_240.0,PAGE_242.0,PAGE_243.0,PAGE_244.0,PAGE_245.0,PAGE_246.0,PAGE_247.0,PAGE_249.0,PAGE_250.0,PAGE_251.0,PAGE_252.0,PAGE_253.0,PAGE_254.0,PAGE_256.0,PAGE_259.0,PAGE_260.0,PAGE_261.0,PAGE_262.0,PAGE_263.0,PAGE_264.0,PAGE_268.0,PAGE_269.0,PAGE_270.0,PAGE_271.0,PAGE_272.0,PAGE_274.0,PAGE_275.0,PAGE_276.0,PAGE_277.0,PAGE_278.0,PAGE_279.0,PAGE_280.0,PAGE_281.0,PAGE_282.0,PAGE_283.0,PAGE_284.0,PAGE_285.0,PAGE_286.0,PAGE_287.0,PAGE_288.0,PAGE_289.0,PAGE_290.0,PAGE_291.0,PAGE_292.0,PAGE_293.0,PAGE_294.0,PAGE_295.0,PAGE_296.0,PAGE_297.0,PAGE_298.0,PAGE_299.0,PAGE_300.0,PAGE_301.0,PAGE_302.0,PAGE_303.0,PAGE_304.0,PAGE_305.0,PAGE_306.0,PAGE_307.0,PAGE_308.0,PAGE_310.0,PAGE_314.0,PAGE_315.0,PAGE_316.0,PAGE_317.0,PAGE_318.0,PAGE_319.0,PAGE_320.0,PAGE_321.0,PAGE_322.0,PAGE_323.0,PAGE_324.0,PAGE_325.0,PAGE_326.0,PAGE_327.0,PAGE_328.0,PAGE_329.0,PAGE_330.0,PAGE_331.0,PAGE_332.0,PAGE_333.0,PAGE_334.0,PAGE_335.0,PAGE_336.0,PAGE_337.0,PAGE_338.0,PAGE_339.0,PAGE_340.0,PAGE_341.0,PAGE_342.0,PAGE_343.0,PAGE_344.0,PAGE_345.0,PAGE_346.0,PAGE_347.0,PAGE_348.0,PAGE_349.0,PAGE_350.0,PAGE_354.0,PAGE_356.0,PAGE_357.0,PAGE_360.0,PAGE_363.0,PAGE_364.0,PAGE_365.0,PAGE_369.0,PAGE_370.0,PAGE_371.0,PAGE_372.0,PAGE_373.0,PAGE_374.0,PAGE_375.0,PAGE_376.0,PAGE_377.0,PAGE_378.0,PAGE_379.0,PAGE_380.0,PAGE_381.0,PAGE_382.0,PAGE_385.0,PAGE_386.0,PAGE_388.0,PAGE_390.0,PAGE_391.0,PAGE_392.0,PAGE_393.0,PAGE_395.0,PAGE_397.0,PAGE_398.0,PAGE_399.0,PAGE_400.0,PAGE_401.0,PAGE_402.0,PAGE_403.0,PAGE_404.0,PAGE_405.0,PAGE_406.0,PAGE_407.0,PAGE_408.0,PAGE_409.0,PAGE_410.0,PAGE_411.0,PAGE_412.0,PAGE_415.0,PAGE_416.0,PAGE_417.0,PAGE_418.0,PAGE_419.0,PAGE_420.0,PAGE_423.0,PAG

In [22]:
def _sort_by_user(df):
    return df.sort_values('USER_ID')

In [47]:
def _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo):
    pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot = _all_pageview_last_adoption_and_last_month(universo, df_pageviews, df_adoption, mes_snapshot, campo)
    
    ratio_preferencia = _ratio_campo_df(pageviews_last_adoption_sum, pageviews_last_month_sum, 'ratio_adoption_preferencia')
    #ratio_interes = _ratio_campo_series(pageviews_last_adoption_sum_tot, pageviews_last_adoption_sum_tot, 'ratio_adoption_interes')

    #df_union = pd.merge(universo, ratio_preferencia, how='left', left_on=['user_id'], right_on=['USER_ID'])
    #df_union = pd.merge(df_union, ratio_interes, how='left', left_on=['user_id'], right_on=['user_id'])

    return ratio_preferencia, pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot #df_union

In [48]:
campo = 'PAGE'
ratio_preferencia, pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot = _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)
#ratio_preferencia, ratio_interes = _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [49]:
ratio_preferencia.head(10)
_sum_campo_user(df_pageviews, campo)

,ratio_adoption_preferencia_PAGE_1.0,ratio_adoption_preferencia_PAGE_10.0,ratio_adoption_preferencia_PAGE_100.0,ratio_adoption_preferencia_PAGE_1000.0,ratio_adoption_preferencia_PAGE_1001.0,ratio_adoption_preferencia_PAGE_1002.0,ratio_adoption_preferencia_PAGE_1004.0,ratio_adoption_preferencia_PAGE_1005.0,ratio_adoption_preferencia_PAGE_1007.0,ratio_adoption_preferencia_PAGE_1009.0,...,ratio_adoption_preferencia_PAGE_988.0,ratio_adoption_preferencia_PAGE_989.0,ratio_adoption_preferencia_PAGE_99.0,ratio_adoption_preferencia_PAGE_990.0,ratio_adoption_preferencia_PAGE_994.0,ratio_adoption_preferencia_PAGE_995.0,ratio_adoption_preferencia_PAGE_997.0,ratio_adoption_preferencia_PAGE_998.0,ratio_adoption_preferencia_PAGE_999.0,ratio_adoption_preferencia_USER_ID
1,1.866667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,NaN,22.000000
2,0.100000,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.500000
3,0.000000,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.666667
4,5.500000,0.758621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.000000
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,20.800000
6,inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,19.166667
7,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,NaN,23.714286
8,inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,24.375000
9,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,22.666667
10,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000


In [ ]:
ratio_interes.head(10)

In [22]:
def ratio_adoption_page_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'PAGE'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_ccategory_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_cctop_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_ccbottom_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_site_id_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'SITE_ID'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_mobile_device_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'IS_MOBILE_DEVICE'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_connection_speed_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONNECTION_SPEED'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_adoption_on_site_search_term_all(universo, df_devicedata, mes_snapshot):
    campo = 'ON_SITE_SEARCH_TERM'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)